In [16]:
import pandas as pd
import numpy as np
import json
import re
import os
import fnmatch
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Problem:
- doc_list or file_list or whatever aku gatau gimana cara ngeread semuanya wkwk soalnya waktu itu gabisa, error
- sampe stemming

In [70]:
""" 1. list folder chatx DONE
    2. list file dalam folder
    
    EXPECTED:
    current dir + /parsed doc/ + chatx/ + sesix.txt
    
"""
folder = os.getcwd() + '/parsed doc/'
chat_folders = [chat_folder for chat_folder in os.listdir(folder) if not chat_folder.startswith('.')]

paths = []
for i in range(len(chat_folders)):
    pattern = str(folder) + chat_folders[i]
    paths.append(pattern)
    
sesi_paths = []
for i in range(len(paths)):
    for roots, dirs, files in os.walk(paths[i]):
        if "checkpoints" not in roots:
            for file in files:
                sesi_paths.append(roots + '/' + file)

In [75]:
chats = [pd.read_csv(sesi, sep='|', names = ['sender', 'messages', 'sentiment']) for sesi in sesi_paths]
chats

,sender,messages,sentiment
0,dosen,"Baik [MHS], Silahkan dikirimkan dokumen bimbi...",1.0
1,mahasiswa,"Baik bapak, terimakasih banyak informasinya",1.0


In [76]:
sender = chats['sender'].values.tolist()
messages = chats['messages'].values.tolist()
sentiment = chats['sentiment'].values.tolist()

In [77]:
def tokenize(msg):
    token = [re.split('\s+', msg[i]) for i, chat in enumerate(msg)]
    return token

token = tokenize(messages)

In [78]:
def casefolding(msg):
    token_lower = [[token.lower() for token in i if token] for i in msg]
    return token_lower

token_lower = casefolding(token)

In [79]:
def cleaning(msg):
    link = r'(www\.[^\s]+)|(https?:\/\/[^\s]+)|(meet\.google\.[^\s]+)'
    email = r'([a-zA-Z0-9\.\_\-]+@+[a-zA-Z0-9.]+)'
    punct = r'[^a-zA-Z0-9\-\s]'
    
    token_clean = []
    for i in msg:
        token_per_chat = []
        for token in i:
            # remove link
            temp = re.sub(link, '', token)
            # remove email
            temp = re.sub(email, '', temp)
            # remove punctuation
            temp = re.sub(punct, '', temp)
            temp = re.sub(r'\-', ' ', temp)
            # remove numbers
            temp = re.sub(r'\b[0-9]+\b\s*', '', temp)
            token_per_chat.append(temp)
        token_clean.append(token_per_chat)
        while('' in token_per_chat): 
            token_per_chat.remove('')
    return token_clean

token_clean = cleaning(token_lower)

In [80]:
def normalize(msg):
    with open('slang_words.json', 'r') as f:
        dict = json.load(f)
    
    normalized_chat = []
    for i in msg:
        slang_dict = {v:k for v, k in dict.items()}
        token_per_chat = []
        for token in i:
            normal = slang_dict.get(token, token)
            token_per_chat.append(normal)
        normalized_chat.append(token_per_chat)
    return normalized_chat

token_normal = normalize(token_clean)

In [81]:
def filtering(msg):
    with open('stopword_list_tala.txt', 'r') as tala:
        stoplist = tala.read()
    token_filtered = [[token for token in i if not token in stoplist] for i in msg]
    return token_filtered

token_filtered = filtering(token_normal)

In [82]:
stemmer = StemmerFactory().create_stemmer()

def stemming(msg):
    token_stemmed = [[stemmer.stem(token) for token in i] for i in msg]
    return token_stemmed

token_stemmed = stemming(token_filtered)

In [ ]:
setToken = [set(token) for token in token_stemmed]
setToken

In [83]:
token_stemmed

[['selamat',
  'pagi',
  'mahasiswa',
  'semester',
  'teknik',
  'informatika',
  'ub',
  'bimbing',
  'skripsi',
  'mohon',
  'bantu',
  'verifikasi',
  'nilai',
  'p2',
  'skripsi',
  'mohon',
  'iya',
  'orangtua',
  'kendala',
  'sulit',
  'bisnis',
  'situasi',
  'covid',
  'lulus',
  'semester',
  'semester',
  'berat',
  'biaya',
  'uang',
  'kuliah',
  'mohon',
  'lanjut',
  'tahap',
  'seminar',
  'hasil',
  'semester',
  'perhati',
  'tolong',
  'terima',
  'kasih',
  'hormat',
  'halomoan',
  'selamat',
  'siang',
  'mahasiswa',
  'semester',
  'teknik',
  'informatika',
  'ub',
  'bimbing',
  'skripsi',
  'mohon',
  'bantu',
  'verifikasi',
  'nilai',
  'p2',
  'skripsi',
  'mohon',
  'orangtua',
  'kendala',
  'sulit',
  'bisnis',
  'situasi',
  'covid',
  'lulus',
  'semester',
  'semester',
  'berat',
  'biaya',
  'uang',
  'kuliah',
  'mohon',
  'lanjut',
  'tahap',
  'seminar',
  'hasil',
  'semester',
  'perhati',
  'tolong',
  'terima',
  'kasih',
  'hormat'],
 ['iy

In [73]:
for i in setToken:
    print(len(i))

37
1
2
5
2
2
5
4
1
